In [2]:
import sxobsplan

from pathlib import Path
import pandas as pd
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import EarthLocation

import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from mpl_toolkits.axes_grid1 import make_axes_locatable
import _rcparams

In [3]:
# SBDB
fpath_sbdb = Path("../data/sbdb_query_results_comet_ver260105.csv") # SBDB comet query results (latest ver)
df_sbdb = pd.read_csv(fpath_sbdb)
df_sbdb.head()

,spkid,full_name,pdes,name,prefix,neo,pha,sats,H,G,...,rms,two_body,A1,A1_sigma,A2,A2_sigma,A3,A3_sigma,DT,DT_sigma
0,1000036,1P/Halley,1P,Halley,P,Y,NaN,0,NaN,NaN,...,0.61032,NaN,4.900000e-10,4.000000e-11,1.600000e-10,4.600000e-15,NaN,NaN,NaN,NaN
1,1000025,2P/Encke,2P,Encke,P,Y,NaN,0,NaN,NaN,...,0.44859,NaN,2.300000e-10,6.100000e-11,-6.500000e-13,4.000000e-12,NaN,NaN,NaN,NaN
2,1000504,3D/Biela,3D,Biela,D,Y,NaN,0,NaN,NaN,...,NaN,NaN,3.900000e-09,NaN,-2.500000e-10,NaN,NaN,NaN,NaN,NaN
3,1000026,4P/Faye,4P,Faye,P,NaN,NaN,0,NaN,NaN,...,0.54329,NaN,4.000000e-09,1.200000e-10,3.200000e-10,6.300000e-11,-7.100000e-10,4.200000e-11,-37.7,2.17
4,1000505,5D/Brorsen,5D,Brorsen,D,Y,NaN,0,NaN,NaN,...,NaN,NaN,1.300000e-08,NaN,1.300000e-09,NaN,NaN,NaN,NaN,NaN


In [4]:
df_sbdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3878 entries, 0 to 3877
Data columns (total 79 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   spkid           3878 non-null   int64  
 1   full_name       3878 non-null   object 
 2   pdes            3878 non-null   object 
 3   name            3769 non-null   object 
 4   prefix          3878 non-null   object 
 5   neo             124 non-null    object 
 6   pha             0 non-null      float64
 7   sats            3878 non-null   int64  
 8   H               1 non-null      float64
 9   G               0 non-null      float64
 10  M1              1867 non-null   float64
 11  M2              826 non-null    float64
 12  K1              1867 non-null   float64
 13  K2              826 non-null    float64
 14  PC              819 non-null    float64
 15  diameter        103 non-null    float64
 16  extent          2 non-null      object 
 17  albedo          19 non-null     f

In [5]:
# Ephemeris
EPHDIR = Path("../eph")

In [6]:
# Observatory locations

# LSGT (Siding Spring Observatory, Australia)
siding_spring_obs = EarthLocation(
    lon=149.0644 * u.deg,  # (149° 03' 52" E)
    lat=-31.2733 * u.deg,  # (31° 16' 24" S)
    height=1165 * u.m
)

# 7DT (el Sauce Observatory, Chile)
el_sauce_obs = EarthLocation(
    lon=-70.763 * u.deg,
    lat=-30.4725 * u.deg,
    height=1600 * u.m
)

# LOAO (Mount Lemmon Observatory, USA)
lemmon_obs = EarthLocation(
    lon=-110.7893 * u.deg,
    lat=32.4420 * u.deg,
    height=2791 * u.m
)

dict_observatory = {
    "gemini_north": "gemini_north",
    "gemini_south": "gemini_south",
    "lemmon"      : lemmon_obs,
    "lsgt"        : siding_spring_obs,
    "sevendt"     : el_sauce_obs
}

for obsname in dict_observatory:
    print(obsname)
    print(sxobsplan.resolve_location(dict_observatory[obsname]))

gemini_north
(-5464283.965730824, -2493783.6435031025, 2150785.9062398607) m
gemini_south
(1820193.0684460273, -5208343.034275673, -3194842.5004834323) m
lemmon
(-1913184.9927148158, -5039329.780596793, 3403394.0181447067) m
lsgt
(-4680991.411618329, 2805467.2896081726, -3292434.228573915) m
sevendt
(1813194.1454216233, -5195977.984234542, -3216438.7522559315) m


In [ ]:
VISDIR = Path("../visibility_ver260105_allcomet")
VISDIR.mkdir(exist_ok=True, parents=True)

for idx, row in df_sbdb.iterrows():

    pdes = row.pdes
    fpath_eph = EPHDIR / f"{''.join(pdes.split())}.csv"
    fpath_vis = VISDIR / f"{''.join(pdes.split())}.csv"

    if not fpath_eph.exists():
        print(f"{fpath_eph.name} does not exists. Skip the rows.")
        continue

    # target info.
    target = df_sbdb[df_sbdb["pdes"] == pdes].iloc[0]

    # ephemeris info.
    eph = pd.read_csv(fpath_eph)

    # Skip if all eph.r > 9 or all eph.Tmag > 22
    if (eph.r > 10).all() or (eph.Tmag > 22).all():
        print(f"{fpath_eph.name}: all r > 9 or all Tmag > 22. Skip the rows.")
        continue

    # Visibility
    df_visible = pd.DataFrame()

    for idx, row in eph.iterrows():

        df_visible.at[idx, "pdes"] = target["pdes"]
        df_visible.at[idx, "full_name"] = target["full_name"]
        df_visible.at[idx, "name"] = target["name"]
        df_visible.at[idx, "class"] = target["class"].upper()

        df_visible.at[idx, "datetime_jd"] = row.datetime_jd
        df_visible.at[idx, "ra"] = row.RA
        df_visible.at[idx, "dec"] = row.DEC
        df_visible.at[idx, "r"] = row.r
        df_visible.at[idx, "delta"] = row.delta
        df_visible.at[idx, "tmag"] = row.Tmag
        df_visible.at[idx, "elong"] = row.elong
        df_visible.at[idx, "lunar_elong"] = row.lunar_elong
        df_visible.at[idx, "glxlat"] = row.GlxLat
        df_visible.at[idx, "alpha"] = row.alpha
        
        for observatory, location in dict_observatory.items():

            is_visible, blocks = sxobsplan.is_target_visible(
                ra=row.RA*u.deg, dec=row.DEC*u.deg, date=Time(row.datetime_jd, format="jd"),
                location=location,
                elev_min=30*u.deg, duration=1*u.hour
            )

            df_visible.at[idx, f"is_visible_{observatory}"] = is_visible

            if is_visible:
                df_visible.at[idx, f"duration_{observatory}"] = blocks[0]["duration"].to_value("hour")
            else:
                df_visible.at[idx, f"duration_{observatory}"] = None

    df_visible.to_csv(fpath_vis)
    print(f"Visibility saved. {fpath_vis}")

1P.csv: all r > 9 or all Tmag > 22. Skip the rows.
Visibility saved. ../visibility_ver260105_comet/2P.csv
3D.csv does not exists. Skip the rows.
Visibility saved. ../visibility_ver260105_comet/4P.csv
5D.csv does not exists. Skip the rows.
Visibility saved. ../visibility_ver260105_comet/6P.csv
Visibility saved. ../visibility_ver260105_comet/7P.csv
8P.csv: all r > 9 or all Tmag > 22. Skip the rows.
Visibility saved. ../visibility_ver260105_comet/9P.csv
Visibility saved. ../visibility_ver260105_comet/10P.csv
Visibility saved. ../visibility_ver260105_comet/11P.csv


OSError: Cannot save file into a non-existent directory: '../visibility_ver260105_comet'

In [7]:
FIGDIR = Path("../fig")
FIGDIR.mkdir(exist_ok=True, parents=True)

for idx, row in df_sbdb.iterrows():

    pdes = row.pdes
    
    fpath_eph = EPHDIR/f"{''.join(pdes.split())}.csv"
    fpath_vis = VISDIR/f"{''.join(pdes.split())}.csv"
    fpath_fig = FIGDIR/f"{''.join(pdes.split())}.png"
    if not fpath_vis.exists():
        print(f"{fpath_vis.name} does not exists. Skip the rows.")
        continue

    df_visible = pd.read_csv(fpath_vis)
    eph = pd.read_csv(fpath_eph)

    # SPHEREX window (80 < e < 110)
    mask_sx = (eph.elong > 80) & (eph.elong < 110)

    fig = plt.figure(figsize=(15, 3))
    ax = fig.add_subplot()

    date_list = pd.to_datetime(df_visible.datetime_jd, unit="D", origin="julian")

    sc_r  = ax.scatter(date_list, [1]*len(date_list), c=eph.r, cmap="jet_r", marker="s", vmin=1, vmax=9)
    sc_v  = ax.scatter(date_list, [2]*len(date_list), c=eph.Tmag, cmap="gray", marker="s", vmin=16, vmax=22)
    ax.axhline(y=3, color="k", linestyle=":")
    sc_sx = ax.scatter(date_list[mask_sx], [4]*sum(mask_sx), c="C3", marker="s")
    observatories = {
        "gemini_north": {"yloc": 5, "color": "C4"},
        "gemini_south": {"yloc": 6, "color": "C5"},
    }
    for observatory, props in observatories.items():
        mask_visible = df_visible[f"is_visible_{observatory}"]
        ax.scatter(date_list[mask_visible], [props["yloc"]]*sum(mask_visible), 
                c=props["color"], marker="s")

    yticks = [1, 2, 4, 5, 6]
    yticklabels = [
        r"$R_\mathrm{h}$ (au)",
        r"$V_\mathrm{mag}$",
        "SPHEREx Window",
        "Gemini North",
        "Gemini South"
    ]
    ax.set_yticks(yticks)
    ax.set_yticklabels(yticklabels)
    ax.set_title(f"{eph['targetname'].iloc[0]}")

    # Colorbars
    divider = make_axes_locatable(ax)
    cax_r = divider.append_axes("right", size="2%", pad=0.2)
    cax_v = divider.append_axes("right", size="2%", pad=0.3)
    cb_r = plt.colorbar(sc_r, cax=cax_r, orientation="vertical")
    # cb_r.set_label(r"$R_\mathrm{h}$ [au]")
    cb_v = plt.colorbar(sc_v, cax=cax_v, orientation="vertical")
    # cb_v.set_label(r"$V_\mathrm{mag}$")

    ax.set_xlim(date_list.min(), date_list.max()+pd.Timedelta(days=5))
    ax.set_ylim(7, 0)
    ax.xaxis.set_minor_locator(mdates.MonthLocator())
    ax.grid(True, axis="x", which="major", linestyle="--", alpha=0.5)
    ax.grid(True, axis="x", which="minor", linestyle=":", alpha=0.3)

    plt.savefig(fpath_fig)
    print(f"Figure saved: {fpath_fig}")
    plt.close()

1P.csv does not exists. Skip the rows.
Figure saved: ../fig/2P.png
3D.csv does not exists. Skip the rows.
Figure saved: ../fig/4P.png
5D.csv does not exists. Skip the rows.
Figure saved: ../fig/6P.png
Figure saved: ../fig/7P.png
8P.csv does not exists. Skip the rows.
Figure saved: ../fig/9P.png
Figure saved: ../fig/10P.png
Figure saved: ../fig/11P.png
Figure saved: ../fig/12P.png
Figure saved: ../fig/13P.png
Figure saved: ../fig/14P.png
Figure saved: ../fig/15P.png
Figure saved: ../fig/16P.png
Figure saved: ../fig/17P.png
18D.csv does not exists. Skip the rows.
Figure saved: ../fig/19P.png
20D.csv does not exists. Skip the rows.
Figure saved: ../fig/21P.png
Figure saved: ../fig/22P.png
23P.csv does not exists. Skip the rows.
Figure saved: ../fig/24P.png
25D.csv does not exists. Skip the rows.
Figure saved: ../fig/26P.png
27P.csv does not exists. Skip the rows.
Figure saved: ../fig/28P.png
Figure saved: ../fig/29P.png
Figure saved: ../fig/30P.png
Figure saved: ../fig/31P.png
Figure save